In [1]:
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np
import os, io
from surprise import SVDpp
from surprise import accuracy
from surprise import Reader, Dataset, SVD, evaluate
from surprise import SVDpp

from surprise.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [28]:
#Reading training dataset
df = pd.read_csv('yelp_restaurant_train.csv')

In [29]:
df.head()

,business_id,date,review_id,text,user_id,attributes,name,city,categories,avg_business_stars,avg_user_star,stars
0,HMSDOjt_KCyknzjQ9aI5Jw,2018-11-04 23:44:27,0HWPTEnVT7L5BCKj33erLQ,"I would like to start with the classic line ""I...",iGtInQDTZ89mKnkhFWdlfA,"{'OutdoorSeating': 'True', 'RestaurantsGoodFor...",Tomo Noodles and Dumplings,Las Vegas,"Noodles, Restaurants, Japanese, Ramen",4.0,3.04,1.0
1,yXiDD18UV49-7UhA6dWjAw,2018-09-07 20:41:42,Y6xCQlzc9YTXD3IKMGN-oQ,We come here for once at least once a month. I...,O3cItff0mKAfXtl5VmbW2w,"{'GoodForMeal': ""{'dessert': False, 'latenight...",Giacomo's Pizzeria and Italian Restaurant,Charlotte,"Sandwiches, Pizza, Italian, Restaurants, Event...",4.0,4.15,4.0
2,VSNUFYBQ_wOFmRXZ8SeQ4w,2015-09-30 02:07:55,9j_uLXkEgpN52Lz5VJ8sMQ,First meal in Pittsburgh was courtesy of S&D! ...,U5YQX_vMl_xQy8EQDqlNQQ,"{'Alcohol': ""u'none'"", 'DogsAllowed': 'False',...",S&D Polish Deli,Pittsburgh,"Polish, Ethnic Food, Specialty Food, Food, Res...",4.5,3.48,4.0
3,lVVJMvqu4LXL5rBqjO6eqg,2015-05-30 07:40:17,xilO0UqpI_EYJg0nzTBqzw,The Pork bone soup is delicious! The broth loo...,JrgMipJRhagq42ROTzC_CQ,"{'HasTV': 'False', 'WiFi': ""u'no'"", 'BusinessP...",Tofu Village - House of Soon Tofu,Toronto,"Restaurants, Korean",4.5,2.53,4.0
4,EEIz44ewHhOKmfTloAK13g,2017-11-19 16:47:26,P5LDWTH6cxQK-_IWtvyOWw,Words cannot express how delectably divine a d...,8lofUN7rFkwT2bw4b5SM4g,"{'RestaurantsPriceRange2': '4', 'RestaurantsTa...",Alo Restaurant,Toronto,"Bars, Nightlife, Restaurants, French",4.5,3.97,5.0


# User Star Ratings

In [5]:
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'business_id', 'avg_user_star']], reader)

In [6]:
trainset, testset = train_test_split(data, test_size=0.20)

In [7]:
bsl_options = {'method': 'als',
               'n_epochs': 7, 
               'reg_u': 3, 
               'reg_i': 7   
               }

In [8]:
from surprise import BaselineOnly

algo = BaselineOnly(bsl_options=bsl_options)
predictions_baseline = algo.fit(trainset).test(testset)
accuracy.rmse(predictions_baseline)

Estimating biases using als...
RMSE: 0.1264


0.1264270484754842

In [9]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose = False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.12563296, 0.12734727, 0.12682037, 0.12629366, 0.12710169]),
 'fit_time': (3.4509639739990234,
  3.5700008869171143,
  3.8514671325683594,
  3.7625977993011475,
  3.895721912384033),
 'test_time': (1.8759520053863525,
  1.7806272506713867,
  1.8156070709228516,
  1.7401683330535889,
  2.370877742767334)}

In [10]:
#Evaluating RMSE of SVD algorithm
svd = SVD()
evaluate(svd, data, measures=['RMSE'])

Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.2271
------------
Fold 2
RMSE: 0.2269
------------
Fold 3
RMSE: 0.2285
------------
Fold 4
RMSE: 0.2275
------------
Fold 5
RMSE: 0.2273
------------
------------
Mean RMSE: 0.2275
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.2270822142246678,
                             0.22691808309721279,
                             0.22847824232339808,
                             0.22752754548074253,
                             0.22731541265987198]})

In [11]:
#Creating training Set
svd.train(trainset)

In [13]:
predictions_svd = svd.test(testset)

In [14]:
accuracy.rmse(predictions_svd, verbose=True)

RMSE: 0.2278


0.22782990343296713

In [18]:
algo = SVDpp(n_factors=150,n_epochs=20)

In [19]:
algo.fit(trainset)

In [20]:
predictions_svdpp = algo.test(testset)

In [21]:
accuracy.rmse(predictions_svdpp, verbose=True)

RMSE: 0.2180


0.2180187569157979

In [23]:
algosvdpp = SVDpp(n_factors=150,n_epochs=40)

In [24]:
algosvdpp.fit(trainset)

In [25]:
predictions_svdpp = algosvdpp.test(testset)

In [26]:
accuracy.rmse(predictions_svdpp, verbose=True)

RMSE: 0.1646


0.16462905899520572

# Business Star Ratings

In [31]:
reader = Reader()
data2 = Dataset.load_from_df(df[['user_id', 'business_id', 'avg_business_stars']], reader)

In [32]:
btrainset, btestset = train_test_split(data2, test_size=0.20)

In [33]:
bsl_options = {'method': 'als',
               'n_epochs': 7, 
               'reg_u': 3, 
               'reg_i': 7   
               }

In [34]:
algo = BaselineOnly(bsl_options=bsl_options)
predictions_baseline = algo.fit(btrainset).test(btestset)
accuracy.rmse(predictions_baseline)

Estimating biases using als...
RMSE: 0.2470


0.247034489729352

In [35]:
cross_validate(algo, data2, measures=['RMSE'], cv=5, verbose = False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.24586841, 0.2441069 , 0.24629341, 0.24755228, 0.24628698]),
 'fit_time': (3.4438629150390625,
  3.9018750190734863,
  4.1725029945373535,
  4.077677965164185,
  4.303909063339233),
 'test_time': (2.4462530612945557,
  2.990434169769287,
  1.5622899532318115,
  2.662278175354004,
  2.9894511699676514)}

In [36]:
#Evaluating RMSE of SVD algorithm
svd = SVD()
evaluate(svd, data2, measures=['RMSE'])

Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.2654
------------
Fold 2
RMSE: 0.2630
------------
Fold 3
RMSE: 0.2626
------------
Fold 4
RMSE: 0.2643
------------
Fold 5
RMSE: 0.2624
------------
------------
Mean RMSE: 0.2635
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.2654421767982772,
                             0.26297453710780183,
                             0.26257294662313757,
                             0.2642893780657295,
                             0.2623721002944852]})

In [37]:
svd.train(btrainset)
predictions_svd = svd.test(btestset)
accuracy.rmse(predictions_svd, verbose=True)

RMSE: 0.2645


0.26452979677741395

In [38]:
algosvdpp = SVDpp(n_factors=150,n_epochs=40)

In [39]:
algosvdpp.fit(btrainset)

In [40]:
predictions_svdpp = algosvdpp.test(btestset)

In [41]:
accuracy.rmse(predictions_svdpp, verbose=True)

RMSE: 0.2312


0.2312354772651732